<a href="https://colab.research.google.com/github/MungaiMwangi001/LLMs/blob/main/ft_gpt3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Installing Depedancies**

In [ ]:
!pip install openai fastapi uvicorn pydantic


**Install OpenAI library and set Api Key**

- The OpenAI key is stored in collabs

---



In [ ]:
import os
from openai import OpenAI

os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"
client = OpenAI()


**Prepare the dataset**
- Data is stored in jsonl fprmat  for OpenAI
- Each line is  a separate  json Object

In [ ]:
import json

data = [
    {
        "messages": [
            {"role": "system", "content": "You are a teaching assistant for Machine Learning."},
            {"role": "user", "content": "What is machine learning?"},
            {"role": "assistant", "content": "'Tis but the art of teaching machines to think..."}
        ]
    }
]

with open("train.jsonl", "w") as f:
    for row in data:
        f.write(json.dumps(row) + "\n")
print("train.jsonl ready")


**Upload the file to OpenAI**

In [ ]:
upload = client.files.create(
    file=open("train.jsonl", "rb"),
    purpose="fine-tune"
)
print("File ID:", upload.id)


FileObject(id='file-EHVfZvuXQ3VFdF56HyBk5x', bytes=245, created_at=1754948566, filename='train.jsonl', object='file', purpose='fine-tune', status='processed', expires_at=None, status_details=None)


**Create a fine-tuned model**

In [ ]:
job = client.fine_tuning.jobs.create(
    training_file=upload.id,
    model="gpt-3.5-turbo"
)
print(job)


**Monitor the job**

In [ ]:
import time

job_id = job.id
while True:
    status = client.fine_tuning.jobs.retrieve(job_id)
    print("Status:", status.status)
    if status.status in ["succeeded", "failed", "cancelled"]:
        print("Final:", status)
        break
    time.sleep(30)


**Test the model**

In [ ]:
ft_model = status.fine_tuned_model
completion = client.chat.completions.create(
    model=ft_model,
    messages=[
        {"role": "system", "content": "You are a teaching assistant for Machine Learning."},
        {"role": "user", "content": "Explain overfitting."}
    ]
)
print(completion.choices[0].message)


***DEPLOYMENT***
**Generate  FASTAPI  + dOCKER SET UP**

In [ ]:
import os

os.makedirs("deployment", exist_ok=True)

with open("deployment/app.py", "w") as f:
    f.write(f'''
from fastapi import FastAPI
from pydantic import BaseModel
from openai import OpenAI
import os

os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"
client = OpenAI()
app = FastAPI()

class Query(BaseModel):
    question: str

@app.post("/ask")
def ask(query: Query):
    response = client.chat.completions.create(
        model="{ft_model}",
        messages=[
            {{"role": "system", "content": "You are a teaching assistant for Machine Learning."}},
            {{"role": "user", "content": query.question}}
        ]
    )
    return {{"answer": response.choices[0].message.content}}
''')

with open("deployment/requirements.txt", "w") as f:
    f.write("fastapi\nuvicorn\nopenai\npydantic\n")

with open("deployment/Dockerfile", "w") as f:
    f.write('''
FROM python:3.11-slim
WORKDIR /app
COPY app.py /app
COPY requirements.txt /app
RUN pip install --no-cache-dir -r requirements.txt
EXPOSE 8000
CMD ["uvicorn", "app:app", "--host", "0.0.0.0", "--port", "8000"]
''')

!cd deployment && zip -r deployment_package.zip .


**Download deployment package**

In [ ]:
from google.colab import files
files.download("deployment/deployment_package.zip")


ERROR:    Error loading ASGI app. Could not import module "app".


**Unzip locally and run this in bash **

- docker build -t my-ft-model .
- docker run -p 8000:8000 my-ft-model